# Lab 20A

1. Given the code below, integrate your code from the previous lab (or put the new def below in your code) so that it displays a button to capture the picture from the drone. The captured picture should appear in the GUI as well after this button is pressed.

In [2]:
from tkinter import *
import cv2
from djitellopy import Tello

# Initialize and connect to Tello
tello = Tello()
tello.connect()

# Start the video stream
tello.streamon()
frame_read = tello.get_frame_read()


def capture_and_display():
    # Capture frame from Tello stream
    frame = frame_read.frame

    # Check if the frame is captured successfully
    if frame is not None and frame.size != 0:
        # Resize frame for display
        frame_resized = cv2.resize(frame, (200, 150))

        # Convert the image to a format Tkinter can display
        _, buffer = cv2.imencode(".ppm", frame_resized)
        image = PhotoImage(data=buffer.tobytes())

        # Display the image in the Tkinter label
        display_label.config(image=image)
        display_label.image = image  # Keep a reference to avoid garbage collection
    else:
        print("Failed to capture frame from the Tello stream.")

# GUI setup
window = Tk()
window.geometry("500x400")



capture_button = Button(text="Capture Photo", command=capture_and_display)
capture_button.place(x=120, y=150, width=120, height=25)

display_label = Label(window)
display_label.place(x=250, y=250, width=200, height=150)

window.mainloop()


[INFO] tello.py - 129 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 438 - Send command: 'command'
[INFO] tello.py - 462 - Response command: 'ok'
[INFO] tello.py - 438 - Send command: 'streamon'
[INFO] tello.py - 462 - Response streamon: 'ok'


2. Place your modified code in the space below. This coide should combine the functioning buttons from the previous lab and the "take a Photo button" from the code above. When the code is run, it should take a photo and display it next to the buttons.

# Section 2- Kernel Killer - This resets the Kernel - Use when Drone program is not responding

In [ ]:
#This program will kill the Kernel causing it to reset. Use when necessary (such as between code executions - when the drone is not responding)
import os
import platform
import subprocess
import sys

def reset_kernel():
    """
    Detect the operating system and reset the Jupyter Notebook kernel accordingly.
    """
    os_name = platform.system()
    print(f"Detected OS: {os_name}")

    if os_name == "Linux":  # Assuming Raspberry Pi 400 runs a Linux-based OS
        try:
            # Run the kill command for Python processes in Linux
            print("Resetting kernel for Pi 400 (Linux)...")
            result = subprocess.run("kill -9 $(ps -A | grep python | awk '{print $1}')", shell=True, check=True)
            print("Kernel reset successfully!")
        except subprocess.CalledProcessError as e:
            print(f"Failed to reset kernel on Pi 400: {e}")
    elif os_name == "Windows":
        try:
            # Use os._exit to exit the Python process on Windows
            print("Resetting kernel for Windows...")
            os._exit(0)
        except Exception as e:
            print(f"Failed to reset kernel on Windows: {e}")
    else:
        print("Unsupported operating system.")

# Run the kernel reset function
reset_kernel()



: 

3. The code below launches the video function of the drone. Try to run it with the drone

In [1]:
from tkinter import *
from tkinter import filedialog
import cv2
from PIL import Image, ImageTk
import pytesseract
from djitellopy import Tello
import threading

# Initialize Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

def analyze_image(image_path):
    """
    Analyzes the image using Tesseract OCR for text recognition.
    """
    try:
        # Perform OCR
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale for better OCR results
        text = pytesseract.image_to_string(gray)
        return text if text.strip() else "No text found in the image."
    except Exception as e:
        print("Error analyzing image:", e)
        return "Error analyzing the image."

def run_tello_video(tello):
    """
    Launches a live video feed from the Tello drone.
    """
    try:
        while True:
            frame = tello.get_frame_read().frame
            if frame is not None:
                cv2.imshow("Tello Drone Live Feed", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to close the video feed
                    break
            else:
                print("No frame available.")
    except Exception as e:
        print("Error during video capture:", e)
    finally:
        cv2.destroyAllWindows()

def run_drone_program():
    """
    Runs the Tello drone program.
    """
    # Initialize Tello and connect
    tello = Tello()
    try:
        tello.connect()
        tello.streamon()
    except Exception as e:
        print("Failed to connect to Tello:", e)
        return run_camera_program()

    frame_read = tello.get_frame_read()

    def takeoff():
        tello.takeoff()
        takeoff_button.config(state=DISABLED)
        land_button.config(state=NORMAL)
        up_button.config(state=NORMAL)
        down_button.config(state=NORMAL)
        left_button.config(state=NORMAL)
        right_button.config(state=NORMAL)
        capture_button.config(state=NORMAL)
        video_button.config(state=NORMAL)

    def land():
        tello.land()
        takeoff_button.config(state=NORMAL)
        land_button.config(state=DISABLED)
        up_button.config(state=DISABLED)
        down_button.config(state=DISABLED)
        left_button.config(state=DISABLED)
        right_button.config(state=DISABLED)
        capture_button.config(state=NORMAL)
        video_button.config(state=DISABLED)

    def up():
        tello.move_up(100)

    def down():
        tello.move_down(100)

    def left():
        tello.move_left(100)

    def right():
        tello.move_right(100)

    def capture_and_analyze():
        # Capture frame from Tello
        frame = frame_read.frame
        if frame is not None:
            image_path = "drone_image.jpg"
            cv2.imwrite(image_path, frame)
            #text = analyze_image(image_path)
            print("Image saved as picture.jpg")        

            # Convert the frame to RGB for display
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(frame_rgb)

            # Resize the image for display in the GUI
            img_pil = img_pil.resize((400, 300))  # Resize as needed
            img_tk = ImageTk.PhotoImage(img_pil)

            display_label.config(image=img_tk)
            display_label.image = img_tk  # Keep a reference to avoid garbage collection
            #text_label.config(text=text)
        else:
            print("Failed to capture image")

    def launch_video():
        # Run live video feed in a separate thread
        threading.Thread(target=run_tello_video, args=(tello,)).start()

    # Drone GUI
    window = Tk()
    window.geometry("500x400")
    window.title("Tello Drone Program")

    global takeoff_button, land_button, up_button, down_button, left_button, right_button, capture_button, video_button

    takeoff_button = Button(window, text="Take Off", command=takeoff, state=NORMAL)
    takeoff_button.place(x=20, y=20, width=120, height=40)

    land_button = Button(window, text="Land", command=land, state=DISABLED)
    land_button.place(x=160, y=20, width=120, height=40)

    up_button = Button(window, text="Up", command=up, state=DISABLED)
    up_button.place(x=90, y=80, width=120, height=40)

    down_button = Button(window, text="Down", command=down, state=DISABLED)
    down_button.place(x=90, y=140, width=120, height=40)

    left_button = Button(window, text="Left", command=left, state=DISABLED)
    left_button.place(x=20, y=200, width=120, height=40)

    right_button = Button(window, text="Right", command=right, state=DISABLED)
    right_button.place(x=160, y=200, width=120, height=40)

    capture_button = Button(window, text="Capture Photo", command=capture_and_analyze, state=NORMAL)
    capture_button.place(x=90, y=260, width=120, height=40)

    video_button = Button(window, text="Launch Video", command=launch_video, state=NORMAL)
    video_button.place(x=90, y=320, width=120, height=40)

    text_label = Label(window, text="", font=("Helvetica", 10), fg="blue", wraplength=300)
    text_label.place(x=250, y=20)

    display_label = Label(window)
    display_label.place(x=350, y=200)

    window.mainloop()

def run_camera_program():
    """
    Runs the fallback program using the USB camera.
    """
    def capture_from_camera():
        cap = cv2.VideoCapture(0)
        ret, frame = cap.read()
        if ret:
            image_path = "camera_image.jpg"
            cv2.imwrite(image_path, frame)
            text = analyze_image(image_path)
            text_label.config(text=text)
        else:
            print("Failed to capture from camera")
        cap.release()

    def upload_image(label):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
        if file_path:
            text = analyze_image(file_path)
            label.config(text=text)

    # Camera GUI
    window = Tk()
    window.geometry("500x400")
    window.title("Camera Program")

    capture_button = Button(window, text="Capture from Camera", command=capture_from_camera)
    capture_button.place(x=90, y=260, width=200, height=40)

    upload_button = Button(window, text="Upload Image", command=lambda: upload_image(text_label))
    upload_button.place(x=90, y=320, width=200, height=40)

    text_label = Label(window, text="", font=("Helvetica", 10), fg="blue", wraplength=300)
    text_label.place(x=20, y=20)

    window.mainloop()

# Main Program Logic
try:
    tello = Tello()
    tello.connect()
    tello.streamon()
    run_drone_program()
except Exception:
    run_camera_program()


[INFO] tello.py - 129 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 438 - Send command: 'command'
[INFO] tello.py - 462 - Response command: 'ok'
[INFO] tello.py - 438 - Send command: 'streamon'
[INFO] tello.py - 462 - Response streamon: 'ok'
[INFO] tello.py - 129 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 438 - Send command: 'command'
[INFO] tello.py - 462 - Response command: 'ok'
[INFO] tello.py - 438 - Send command: 'streamon'
[INFO] tello.py - 462 - Response streamon: 'ok'


Image saved as picture.jpg


[INFO] tello.py - 438 - Send command: 'streamoff'
[INFO] tello.py - 462 - Response streamoff: 'error'
[INFO] tello.py - 438 - Send command: 'streamoff'
[INFO] tello.py - 462 - Response streamoff: 'error'
[INFO] tello.py - 438 - Send command: 'streamoff'
[INFO] tello.py - 462 - Response streamoff: 'error'


4. Run the program above and comment on what happens: